### Simple example
This code shows as simple way to:
- Read in a .nc file using xarray
- Calculate annual values
- Plot global time series
- Plot spatial maps

In [27]:
import utils
import xarray as xr
import esmlab

suff = 'lnd/proc/tseries/month_1/'
path = '/glade/p/cgd/tss/people/oleson/CLM_LAND_ONLY_RELEASE/CLM5/clm50_r270_1deg_GSWP3V1_iso_newpopd_hist/'+suff
file = path + 'clm50_r270_1deg_GSWP3V1_iso_newpopd_hist.clm2.h0.GPP.185001-201412.nc'
file

'/glade/p/cgd/tss/people/oleson/CLM_LAND_ONLY_RELEASE/CLM5/clm50_r270_1deg_GSWP3V1_iso_newpopd_hist/lnd/proc/tseries/month_1/clm50_r270_1deg_GSWP3V1_iso_newpopd_hist.clm2.h0.GPP.185001-201412.nc'

In [55]:
# wrapping the time_set_mid utility corrects time being read in by xr.open_dataset
# data sets have lots of coords, variables, and attributes
ds   = utils.time_set_mid(xr.open_dataset(file, decode_times=True), 'time')
area = ds.area
landfrac = ds.landfrac

In [56]:
# make weights for global sum
weights = area * landfrac 
weights = weights / weights.sum()
weights

<xarray.DataArray (lat: 192, lon: 288)>
array([[2.012404e-07, 2.012404e-07, 2.012404e-07, ..., 2.012404e-07,
        2.012404e-07, 2.012404e-07],
       [1.609842e-06, 1.609842e-06, 1.609842e-06, ..., 1.609842e-06,
        1.609842e-06, 1.609842e-06],
       [3.219248e-06, 3.219248e-06, 3.219248e-06, ..., 3.219248e-06,
        3.219248e-06, 3.219248e-06],
       ...,
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.25 357.5 358.75
Attributes:
    long_name:  grid cell areas
    units:      km^2

--------------------
#### Annual averages, *unweighted here for days/month*
doing this on the full dataset is really slow, not sure why?

In [40]:
# ds_ann = ds.resample(time='AS').mean('time')
# ds_ann

In [51]:
# data arrays strip out only the relevent coords, and attributes
# this is much faster than resampling the full dataset, but we lose landfrac and area
da = ds.GPP
da_ann = da.to_dataset().resample(time='AS').mean('time')
da_ann

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 165)
Coordinates:
  * time     (time) object 1850-01-01 00:00:00 ... 2014-01-01 00:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    GPP      (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 165)
Coordinates:
  * time     (time) object 1850-01-01 00:00:00 ... 2014-01-01 00:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    GPP      (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan

In [ ]:
d = xr.decode_cf(da_ann)

<xarray.DataArray 'GPP' (time: 165, lat: 192, lon: 288)>
[9123840 values with dtype=float32]
Coordinates:
  * time     (time) object 1850-01-01 00:00:00 ... 2014-01-01 00:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:     gross primary production
    units:         gC/m^2/s
    cell_methods:  time: mean

In [11]:
ds_ann = esmlab.resample(d, freq='ann')
ds_ann

/ncar/usr/jupyterhub/envs/pangeo-2019.09.12/lib/python3.7/site-packages/xarray/core/nanops.py:160: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


ValueError: could not convert string to float: b'02/08/18'

In [15]:
d.to_datetime()

AttributeError: 'Dataset' object has no attribute 'to_datetime'